<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.1/533.1 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=e251ebe1c942c943b48f373473bb1ab7ba4d2e2a3dce38c72aea98f94030a100
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-28 10:44:25
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.35 C
-------------------
Today PnL: 28.07 K (0.21%)
Current PnL: -33.83 L (-21.57%)
CY Booked + Current PnL: -19.54 L (-12.46%)
-------------------
Total profit:  1.37 L
Total loss:  -35.19 L
-------------------
Total Booked + Current PnL: 7.44 L (5.73%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.57%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 99.17 L (73.32%)
Deployed:  1.30 C
Current:  1.35 C
CAGR/XIRR %: 2.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,4.00,8.15,10.24,19.23,16663.0,12266.0,162726.0,27.19,66.0,L-LC,11.55,182.0,0.74,1.20,31.16,XY25,ATH,MINING
77,TTKPRESTIG,770.00,1.91,-22.66,29.41,0.09,22924.0,-22832.0,77945.0,81.00,45.0,M-SC,5.97,253.0,-1.00,0.58,2.71,OX40N,NTT,DURABLES
51,NESTLEIND,1377.00,-2.11,12.47,7.93,21.40,23674.0,33111.0,298537.0,4.42,45.0,X-LC,7.21,14.0,1.40,2.21,19.59,XY25,NTT,FMCG
49,MASFIN,398.61,-0.22,-3.95,27.07,22.05,25476.0,-3870.0,94110.0,-17.34,50.0,H-SC,3.77,168.0,-0.15,0.70,36.80,XR,ATH,FINANCE
32,HCLTECH,1908.19,-0.24,11.37,11.19,23.83,30149.0,27504.0,269428.0,16.15,65.0,X-LC,7.01,13.0,0.91,1.99,29.27,X40,ATH,IT


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,4.12,-10.07,61.87,45.57,145563.0,-26347.0,235272.0,-42.19,46.0,H-MC,7.43,121.0,-0.18,1.74,4.48,AR,NTT,ELECTRICAL
17,COALINDIA,484.83,4.00,8.15,10.24,19.23,16663.0,12266.0,162726.0,27.19,66.0,L-LC,11.55,182.0,0.74,1.20,31.16,XY25,ATH,MINING
71,TANLA,1943.92,3.76,-42.93,284.93,119.69,440302.0,-116231.0,154530.0,-42.51,51.0,H-SC,17.22,150.0,-0.26,1.14,22.08,AR,ATH,IT
58,RELAXO,1176.00,3.14,-51.08,211.32,52.31,150073.0,-74143.0,71017.0,-47.92,42.0,X-SC,10.65,92.0,-0.49,0.53,5.61,X40N,NTT,FOOTWEAR
14,CAMPUS,393.00,2.93,-14.81,56.45,33.28,80686.0,-24842.0,142933.0,-31.36,44.0,M-SC,5.49,221.0,-0.31,1.06,10.78,XY24,NTT,FOOTWEAR


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3460.25,-5.44,-13.31,39.52,20.95,86252.0,-33519.0,218249.0,-14.13,26.0,X-LC,11.72,36.0,-0.39,1.61,17.58,X40,ATH,PAINTS
44,JCHAC,2282.00,-4.28,-43.56,77.17,-0.01,55661.0,-55677.0,72128.0,14660.87,28.0,M-SC,14.94,235.0,-1.00,0.53,0.00,OX40N,BTT,AC
51,NESTLEIND,1377.00,-2.11,12.47,7.93,21.40,23674.0,33111.0,298537.0,4.42,45.0,X-LC,7.21,14.0,1.40,2.21,19.59,XY25,NTT,FMCG
33,HINDUNILVR,2922.00,-2.06,-8.52,24.26,13.67,57047.0,-21913.0,235150.0,-15.44,47.0,X-LC,2.04,9.0,-0.38,1.74,11.01,XY25,NTT,FMCG
12,BERGEPAINT,680.00,-1.76,-13.31,39.75,21.14,78337.0,-30266.0,197073.0,-19.45,26.0,X-MC,5.52,75.0,-0.39,1.46,6.77,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,0.63,-0.85,21.77,20.74,43853.0,-1732.0,201438.0,31.15,52.0,M-MC,4.79,189.0,-0.04,1.49,9.67,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.62,-10.47,132.22,107.91,174782.0,-15454.0,132190.0,-27.04,34.0,M-SC,10.45,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
72,TATAELXSI,9161.00,-0.76,-22.55,70.47,32.03,71954.0,-29729.0,102106.0,-16.20,48.0,H-SC,6.39,157.0,-0.41,0.76,14.37,OX40N,NTT,IT
77,TTKPRESTIG,770.00,1.91,-22.66,29.41,0.09,22924.0,-22832.0,77945.0,81.00,45.0,M-SC,5.97,253.0,-1.00,0.58,2.71,OX40N,NTT,DURABLES
66,SIS,528.00,0.91,-24.66,60.95,21.26,50987.0,-27379.0,83653.0,1973.06,48.0,H-SC,5.12,163.0,-0.54,0.62,13.02,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.16,-24.93,51.16,13.47,103567.0,-67230.0,202437.0,-69.09,40.0,H-SC,5.02,159.0,-0.65,1.50,2.93,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,4.0,8.15,10.24,19.23,16663.0,12266.0,162726.0,27.19,66.0,L-LC,11.55,182.0,0.74,1.2,31.16,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.57,-3.14,77.85,72.27,127771.0,-5324.0,164124.0,-16.97,37.0,M-LC,2.95,101.0,-0.04,1.21,5.49,XR,NTT,IT
38,INDIAMART,4810.62,-0.44,-6.47,121.02,106.72,139609.0,-7976.0,115360.0,-53.68,47.0,H-SC,8.90,138.0,-0.06,0.85,15.40,AR,ATH,MISC
25,FINCABLES,1641.55,-0.62,-10.47,132.22,107.91,174782.0,-15454.0,132190.0,-27.04,34.0,M-SC,10.45,220.0,-0.09,0.98,0.00,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,-0.55,-6.32,44.77,35.63,87900.0,-13238.0,196336.0,-19.40,44.0,H-MC,5.18,119.0,-0.15,1.45,9.30,AR,ATH,PHARMA
49,MASFIN,398.61,-0.22,-3.95,27.07,22.05,25476.0,-3870.0,94110.0,-17.34,50.0,H-SC,3.77,168.0,-0.15,0.70,36.80,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.53,-33.08,134.89,57.19,294396.0,-107878.0,218249.0,-68.35,16.0,X-MC,14.88,58.0,-0.37,1.61,0.00,XY24,NTT,FMCG
43,ITC,452.00,0.41,-21.58,41.27,10.78,77774.0,-51867.0,188451.0,-53.41,17.0,X-LC,16.40,1.0,-0.67,1.39,0.41,X40,NTT,FMCG
52,PAGEIND,51605.07,-0.66,-18.23,59.74,30.62,115794.0,-43224.0,193830.0,-41.90,20.0,X-MC,19.22,57.0,-0.37,1.43,0.00,X40,ATH,APPARELS
22,DIXON,18931.72,-1.35,-30.34,88.43,31.26,142153.0,-70016.0,160752.0,-67.43,21.0,X-MC,26.16,56.0,-0.49,1.19,0.00,X40N,ATH,IT
11,BATAINDIA,2096.00,1.19,-44.66,144.33,35.20,102766.0,-57468.0,71202.0,-6.71,22.0,X-SC,19.35,93.0,-0.56,0.53,1.19,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.59,1.01,-12.29,35.17,18.55,116673.0,-46498.0,331739.0,-24.60,54.0,X-LC,0.67,3.0,-0.40,2.45,12.86,X40,ATH,IT
33,HINDUNILVR,2922.00,-2.06,-8.52,24.26,13.67,57047.0,-21913.0,235150.0,-15.44,47.0,X-LC,2.04,9.0,-0.38,1.74,11.01,XY25,NTT,FMCG
81,VBL,671.64,-1.56,-6.21,44.66,35.67,132293.0,-19619.0,296223.0,-17.33,37.0,X-LC,2.38,5.0,-0.15,2.19,6.78,X40N,ATH,FMCG
35,ICICIGI,2252.93,1.44,-2.96,22.98,19.34,44623.0,-5928.0,194181.0,-23.33,37.0,X-MC,2.46,71.0,-0.13,1.44,12.50,X40,ATH,INSURANCE
20,DABUR,735.00,-1.52,-1.15,45.37,43.70,111254.0,-2852.0,245216.0,-9.24,45.0,X-MC,3.16,73.0,-0.03,1.81,13.73,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.53,-33.08,134.89,57.19,294396.0,-107878.0,218249.0,-68.35,16.0,X-MC,14.88,58.0,-0.37,1.61,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,-1.35,-30.34,88.43,31.26,142153.0,-70016.0,160752.0,-67.43,21.0,X-MC,26.16,56.0,-0.49,1.19,0.00,X40N,ATH,IT
31,HAVELLS,2062.85,-0.20,-17.64,60.37,32.08,163073.0,-57868.0,270123.0,-21.86,26.0,X-MC,6.55,69.0,-0.35,2.00,0.00,X40,ATH,ELECTRICAL
52,PAGEIND,51605.07,-0.66,-18.23,59.74,30.62,115794.0,-43224.0,193830.0,-41.90,20.0,X-MC,19.22,57.0,-0.37,1.43,0.00,X40,ATH,APPARELS
53,PGHH,17907.65,-0.34,-10.41,49.78,34.19,95227.0,-22224.0,191296.0,-36.94,31.0,X-MC,6.60,60.0,-0.23,1.41,0.25,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,1.97,-33.26,114.04,42.84,49473.0,-21624.0,43382.0,-56.47,39.0,X-SC,22.76,83.0,-0.44,0.32,1.97,XY24,NTT,MISC
11,BATAINDIA,2096.00,1.19,-44.66,144.33,35.20,102766.0,-57468.0,71202.0,-6.71,22.0,X-SC,19.35,93.0,-0.56,0.53,1.19,X40,NTT,FOOTWEAR
58,RELAXO,1176.00,3.14,-51.08,211.32,52.31,150073.0,-74143.0,71017.0,-47.92,42.0,X-SC,10.65,92.0,-0.49,0.53,5.61,X40N,NTT,FOOTWEAR
50,MEDANTA,1486.00,0.57,-12.42,41.67,24.08,46330.0,-15767.0,111183.0,-19.45,33.0,X-SC,8.64,91.0,-0.34,0.82,3.10,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,1.98,-22.88,85.85,43.32,107828.0,-37272.0,125600.0,-33.73,33.0,X-SC,8.95,90.0,-0.35,0.93,1.98,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.41,-21.58,41.27,10.78,77774.0,-51867.0,188451.0,-53.41,17.0,X-LC,16.40,1.0,-0.67,1.39,0.41,X40,NTT,FMCG
75,TMPV,600.00,0.04,-30.17,76.11,22.98,120059.0,-68154.0,157744.0,-27.96,38.0,X-LC,4.27,2.0,-0.57,1.17,0.84,XY24,NTT,AUTO
73,TCS,4311.59,1.01,-12.29,35.17,18.55,116673.0,-46498.0,331739.0,-24.60,54.0,X-LC,0.67,3.0,-0.40,2.45,12.86,X40,ATH,IT
81,VBL,671.64,-1.56,-6.21,44.66,35.67,132293.0,-19619.0,296223.0,-17.33,37.0,X-LC,2.38,5.0,-0.15,2.19,6.78,X40N,ATH,FMCG
41,INFY,1972.00,-1.11,13.28,18.50,34.24,65574.0,41565.0,354453.0,-11.70,57.0,X-LC,7.31,6.0,0.63,2.62,22.67,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-1.21,-34.74,103.64,32.89,52216.0,-26824.0,50382.0,-722.51,47.0,L-MC,11.22,259.0,-0.51,0.37,38.75,XR,NTT,BANKS
17,COALINDIA,484.83,4.00,8.15,10.24,19.23,16663.0,12266.0,162726.0,27.19,66.0,L-LC,11.55,182.0,0.74,1.20,31.16,XY25,ATH,MINING
32,HCLTECH,1908.19,-0.24,11.37,11.19,23.83,30149.0,27504.0,269428.0,16.15,65.0,X-LC,7.01,13.0,0.91,1.99,29.27,X40,ATH,IT
41,INFY,1972.00,-1.11,13.28,18.50,34.24,65574.0,41565.0,354453.0,-11.70,57.0,X-LC,7.31,6.0,0.63,2.62,22.67,X40,NTT,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,4.00,8.15,10.24,19.23,16663.0,12266.0,162726.0,27.19,66.0,L-LC,11.55,182.0,0.74,1.20,31.16,XY25,ATH,MINING
62,SATIN,274.00,1.61,-13.59,75.71,51.83,180753.0,-37540.0,238744.0,-22.47,71.0,H-SC,3.15,148.0,-0.21,1.77,16.30,XY24,NTT,FINANCE
32,HCLTECH,1908.19,-0.24,11.37,11.19,23.83,30149.0,27504.0,269428.0,16.15,65.0,X-LC,7.01,13.0,0.91,1.99,29.27,X40,ATH,IT
40,INDUSINDBK,1800.00,-1.21,-34.74,103.64,32.89,52216.0,-26824.0,50382.0,-722.51,47.0,L-MC,11.22,259.0,-0.51,0.37,38.75,XR,NTT,BANKS
10,BANDHANBNK,400.00,2.57,-26.47,161.69,92.42,331172.0,-73740.0,204819.0,44.40,62.0,H-SC,10.42,169.0,-0.22,1.51,18.37,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.58
1,25,44.77
2,50,76.93


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
SC,42.63
MC,30.61
LC,26.78


In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Criteria,
XY24,26.29
X40,24.14
X40N,12.69
XY25,11.02
XR,8.77
AR,8.72
OX40N,7.45
SR,0.94


In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

,CurrAlloc%
Conviction,
X-MC,24.09
X-LC,23.29
H-SC,23.13
M-SC,10.91
X-SC,7.83
H-MC,4.66
M-MC,1.49
M-LC,1.21
L-LC,1.20


In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.31,-12.45,49.35
IT,13.04,-21.23,81.79
FINANCE,11.15,-18.38,68.06
MISC,7.02,-36.80,92.76
ELECTRICAL,5.75,-20.88,64.98
PAINTS,5.53,-23.47,41.81
INSURANCE,4.74,-5.94,42.19
PHARMA,4.08,-6.44,39.94
BANKS,2.87,-37.55,120.89


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3416656.0,21
AR,1391675.0,10
XR,1306981.0,12
X40,1243147.0,15
X40N,1027144.0,9
OX40N,786782.0,10
XY25,451111.0,7
SR,293333.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3671016.0,24
X-MC,1774030.0,16
M-SC,1582802.0,15
X-LC,1062035.0,13
X-SC,861588.0,8
H-MC,448105.0,3
L-SC,188650.0,2
M-LC,127771.0,1
H-LC,88100.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1267180.0      6
           AR         938279.0      5
M-SC       XY24       898067.0      6
H-SC       XR         840992.0      7
X-MC       X40        598347.0      7
           XY24       483987.0      3
           X40N       453279.0      4
X-LC       X40        440531.0      6
M-SC       OX40N      361213.0      5
X-SC       X40N       339573.0      3
H-SC       OX40N      331232.0      4
X-SC       XY24       317746.0      3
H-SC       SR         293333.0      2
X-MC       XY25       238417.0      2
X-LC       XY24       235034.0      2
           X40N       234292.0      2
H-MC       AR         233463.0      2
           XY24       214642.0      1
X-SC       X40        204269.0      2
M-SC       XR         191689.0      2
X-LC       XY25       152178.0      3
M-SC       AR         131833.0      2
M-LC       XR         127771.0      1
L-SC       OX40N       94337.0      1
           XR          94313.0      1
H-LC       AR          88100.0      1
L-MC       XR          52216.0      1
M-MC       XY25        43853.0      1
L-LC       XY25        16663.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
